In [3]:
import os
import markdown
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle

# Authenticate and set up Google Docs API
SCOPES = ['https://www.googleapis.com/auth/documents']

def authenticate():
    auth.authenticate_user()
    credentials = None

    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)

    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        # else:
        #     flow = InstalledAppFlow.from_client_secrets_file(
        #         'client_secrets.json', SCOPES)
        #     credentials = flow.run_local_server(port=0)

        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    return build('docs', 'v1', credentials=credentials)

# Parse markdown content
def parse_markdown(md_content):
    html = markdown.markdown(md_content)
    return html

# Create a new Google Doc
def create_google_doc(service, title, content):
    doc = service.documents().create(body={"title": title}).execute()
    document_id = doc.get('documentId')

    requests = []
    requests.append({
        'insertText': {
            'location': {
                'index': 1
            },
            'text': content
        }
    })

    service.documents().batchUpdate(
        documentId=document_id, body={"requests": requests}).execute()
    return f"Document created successfully: https://docs.google.com/document/d/{document_id}"

# Markdown content
markdown_content = """# Product Team Sync - May 15, 2023

## Attendees

- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items

- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps

* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes

* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---

Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

def main():
    service = authenticate()
    parsed_content = parse_markdown(markdown_content)
    print(create_google_doc(service, "Product Team Sync Notes", parsed_content))

if __name__ == "__main__":
    main()


Document created successfully: https://docs.google.com/document/d/1w-7-r_xx0lyotbUXRqvrPJHb8Bc97pHQUrjqVew0k5E
